In [1]:
import numpy as np
import math 
import cmath
import abipy
from abipy.electrons.scr import ScrFile
import time
import matplotlib.pyplot as plt
import pointcloud as pc
from scipy.interpolate import RegularGridInterpolator
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
import Fourier_tool as Ft
import XGChi0 
import MatrixCharacteristic
from abipy import abilab
import abipy.data as abidata
from abipy.abilab import Structure
import random

In [2]:
filename_prim = "Al_Bulk_nkpt4o_DS4_SUS.nc"
Matrr_prim = "Al_D_prim_SUS_ngkpt4_ecuteps17_nband200.npy"

In [4]:
filename_slab = "Al_Slab5_Chi0o_DS1_SUS.nc"
Matrr_slab = "chi0rr_slab5_2.npy"

In [5]:
ngkpt_prim = [4, 4, 4] #has to be entered manually from the data given above
ngkpt_slab = [4, 4, 1]
nsym_var = 0

# Structures

In [6]:
structure_prim=abipy.core.structure.Structure.from_file(filename_prim)
structure_prim

Structure Summary
Lattice
    abc : 2.8535866547158504 2.8535866547158504 2.8535866547158504
 angles : 59.999999999999986 59.99999999999999 59.99999999999999
 volume : 16.430780655166938
      A : 1.6475190232561199 0.0 2.3299437469564794
      B : -0.8237595116280599 1.426793327357925 2.3299437469564794
      C : -0.8237595116280599 -1.426793327357925 2.3299437469564794
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [7]:
structure_slab=abipy.core.structure.Structure.from_file(filename_slab)
structure_slab

Structure Summary
Lattice
    abc : 2.8535866547158504 2.8535866547158504 20.17790474253013
 angles : 90.0 90.0 90.0
 volume : 164.30780655166944
      A : 2.017790474253013 -2.017790474253013 0.0
      B : 2.017790474253013 2.017790474253013 0.0
      C : 0.0 0.0 20.17790474253013
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (0.0000, 2.0178, 2.0178) [-0.5000, 0.5000, 0.1000]
PeriodicSite: Al (0.0000, 0.0000, 4.0356) [0.0000, 0.0000, 0.2000]
PeriodicSite: Al (0.0000, 2.0178, -2.0178) [-0.5000, 0.5000, -0.1000]
PeriodicSite: Al (0.0000, 0.0000, -4.0356) [0.0000, 0.0000, -0.2000]

In [8]:
dict_struct_prim = structure_prim.to_abivars()
natom_prim = dict_struct_prim["natom"]
xred_prim = dict_struct_prim["xred"]
lattice_prim = structure_prim.lattice.matrix
A_prim, B_prim, C_prim = lattice_prim[0], lattice_prim[1], lattice_prim[2]

In [9]:
dict_struct_slab = structure_slab.to_abivars()
natom_slab = dict_struct_slab["natom"]
xred_slab = dict_struct_slab["xred"]
lattice_slab = structure_slab.lattice.matrix
A_slab, B_slab, C_slab = lattice_slab[0], lattice_slab[1], lattice_slab[2]

In [10]:
XGChi0.Supercell_Vis(A_prim, B_prim, C_prim, ngkpt_prim, natom_prim, xred_prim)

In [11]:
XGChi0.Supercell_Vis(A_slab, B_slab, C_slab, ngkpt_slab, natom_slab, xred_slab)

# Loading Reciprocal Space

In [13]:
chi0GG_prim, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk = XGChi0.Build_Chi0GG("Al_Bulk_nkpt4o_DS4_SUS.nc", "FromSymTest", omega = 0)

48
64
{0: (0.0, 0.0, 0.0), 1: (1.0, 0.0, 0.0), 2: (2.0, 0.0, 0.0), 3: (1.0, 1.0, 0.0), 4: (2.0, 1.0, 0.0), 5: (-1.0, 1.0, 0.0), 6: (2.0, 2.0, 0.0), 7: (-1.0, 2.0, 1.0), 8: (-1.0, 0.0, 0.0), 9: (-1.0, -1.0, 0.0), 10: (2.0, -1.0, 0.0), 11: (1.0, -1.0, 0.0), 12: (1.0, 2.0, -1.0), 13: (1.0, 1.0, 1.0), 14: (2.0, 2.0, 2.0), 15: (0.0, 1.0, 1.0), 16: (1.0, 2.0, 2.0), 17: (2.0, -1.0, -1.0), 18: (0.0, 2.0, 2.0), 19: (-1.0, -1.0, -1.0), 20: (0.0, -1.0, -1.0), 21: (-1.0, 2.0, 2.0), 22: (2.0, 1.0, 1.0), 23: (0.0, 0.0, 1.0), 24: (0.0, 0.0, 2.0), 25: (-1.0, -1.0, 1.0), 26: (-1.0, -1.0, 2.0), 27: (0.0, 0.0, -1.0), 28: (1.0, 1.0, -1.0), 29: (1.0, 1.0, 2.0), 30: (0.0, -1.0, 0.0), 31: (0.0, 2.0, 0.0), 32: (0.0, 2.0, -1.0), 33: (0.0, 1.0, -1.0), 34: (0.0, 1.0, 0.0), 35: (0.0, 2.0, 1.0), 36: (0.0, -1.0, 1.0), 37: (2.0, 2.0, -1.0), 38: (2.0, 2.0, 1.0), 39: (-1.0, 1.0, 1.0), 40: (1.0, -1.0, -1.0), 41: (1.0, 2.0, 0.0), 42: (-1.0, 2.0, 0.0), 43: (0.0, -1.0, 2.0), 44: (0.0, 1.0, 2.0), 45: (1.0, 0.0, 1.0), 46: (

In [14]:
chi0GG_slab = np.load("chi0GG_slab5.npy")

# Prim Information Reciprocal Space

In [15]:
chi0GG_const_prim = chi0GG_prim[0, 0, 0]
sum_chi0GG_prim = np.sum(chi0GG_prim)
max_r_chi0GG_prim = np.max(np.abs(np.real(chi0GG_prim)))
max_i_chi0GG_prim = np.max(np.abs(np.imag(chi0GG_prim)))

In [17]:
print("The constant component = ", chi0GG_const_prim,"\nThe sum of all the components = ", sum_chi0GG_prim,"\nThe maximum absolute real value = ", max_r_chi0GG_prim, "\nThe maximum absolute imaginary value = ", max_i_chi0GG_prim,"\nThe shape of the matrix = ", chi0GG_prim.shape)

The constant component =  (-1.258613906429673e-07+1.4504860652603773e-25j) 
The sum of all the components =  (-25.60394124483654+1.0143866257869921e-08j) 
The maximum absolute real value =  0.13829785585403442 
The maximum absolute imaginary value =  5.398534312917036e-07 
The shape of the matrix =  (64, 339, 339)


# Slab Information Reciprocal Space

In [18]:
chi0GG_const_slab = chi0GG_slab[0, 0, 0]
sum_chi0GG_slab = np.sum(chi0GG_slab)
max_r_chi0GG_slab = np.max(np.abs(np.real(chi0GG_slab)))
max_i_chi0GG_slab = np.max(np.abs(np.imag(chi0GG_slab)))

In [22]:
print("The constant component = ", chi0GG_const_slab,"\nThe sum of all the components = ", sum_chi0GG_slab,"\nThe maximum absolute real value = ", max_r_chi0GG_slab, "\nThe maximum absolute imaginary value = ", max_i_chi0GG_slab,"\nThe shape of the matrix = ", chi0GG_slab.shape)

The constant component =  (-8.726038558393157e-09-3.718962296668246e-29j) 
The sum of all the components =  (-0.7640257295835131+9.932670564351666e-06j) 
The maximum absolute real value =  0.05861315503716469 
The maximum absolute imaginary value =  2.4609980755485594e-06 
The shape of the matrix =  (16, 2837, 2837)


# Loading Direct Space

In [23]:
chi0rr_prim = np.load(Matrr_prim)
chi0rr_slab = np.load(Matrr_slab)

# Prim Information Direct Space

In [24]:
chi0rr_origin_prim = chi0rr_prim[0, 0, 0, 0, 0, 0]
sum_chi0rr_prim = np.sum(chi0rr_prim)
sum_per_pert_prim = sum_chi0rr_prim/chi0rr_prim.size
max_r_chi0rr_prim = np.max(np.abs(np.real(chi0rr_prim)))
max_i_chi0rr_prim = np.max(np.abs(np.imag(chi0rr_prim)))

In [28]:
print("The origin component = ", chi0rr_origin_prim,"\nThe sum of all the components = ", sum_chi0rr_prim, "\nThe average variation of the density for a given perturbation of 1 =", sum_per_pert_prim, "\nThe maximum absolute real value = ", max_r_chi0rr_prim, "\nThe maximum absolute imaginary value = ", max_i_chi0rr_prim,"\nThe shape of the matrix = ", chi0rr_prim.shape)

The origin component =  (-25.603941244836573+0j) 
The sum of all the components =  (-5.8721890418369185-5.560135685200862e-14j) 
The average variation of the density for a given perturbation of 1 = (-1.2586139064293806e-07-1.1917300422669887e-21j) 
The maximum absolute real value =  226.27491250387985 
The maximum absolute imaginary value =  0.0004367077655874372 
The shape of the matrix =  (9, 9, 9, 40, 40, 40)


# Slab Information Direct Space

In [25]:
chi0rr_origin_slab = chi0rr_slab[0, 0, 0, 0, 0, 0]
sum_chi0rr_slab = np.sum(chi0rr_slab)
sum_per_pert_slab = sum_chi0rr_slab/chi0rr_slab.size
max_r_chi0rr_slab = np.max(np.abs(np.real(chi0rr_slab)))
max_i_chi0rr_slab = np.max(np.abs(np.imag(chi0rr_slab)))

In [29]:
print("The origin component = ", chi0rr_origin_slab,"\nThe sum of all the components = ", sum_chi0rr_slab, "\nThe average variation of the density for a given perturbation of 1 =", sum_per_pert_slab, "\nThe maximum absolute real value = ", max_r_chi0rr_slab, "\nThe maximum absolute imaginary value = ", max_i_chi0rr_slab,"\nThe shape of the matrix = ", chi0rr_slab.shape)

The origin component =  (-0.7640257295835089+0j) 
The sum of all the components =  (-5.862557591613154-7.162928236659205e-12j) 
The average variation of the density for a given perturbation of 1 = (-8.726038558237648e-09-1.066155632695093e-20j) 
The maximum absolute real value =  307.1598363629626 
The maximum absolute imaginary value =  8.361826076984167e-05 
The shape of the matrix =  (9, 9, 72, 40, 40, 72)


# Visualisation tools 1D

In [30]:
n1_go_prim, n2_go_prim, n3_go_prim, n4_go_prim, n5_go_prim, n6_go_prim = chi0rr_prim.shape

In [31]:
n1_go_slab, n2_go_slab, n3_go_slab, n4_go_slab, n5_go_slab, n6_go_slab = chi0rr_slab.shape

In [41]:
R1D_prim = [0, 0, 0]

In [42]:
X1_prim = np.linspace(0, ngkpt_prim[0], n4_go_prim,endpoint = False)
X2_prim = np.linspace(0, ngkpt_prim[1], n5_go_prim,endpoint = False)
X3_prim = np.linspace(0, ngkpt_prim[2], n6_go_prim,endpoint = False)

X1R_prim = np.linspace(0, 1, n1_go_prim, endpoint = False)
X2R_prim = np.linspace(0, 1, n2_go_prim, endpoint = False)
X3R_prim = np.linspace(0, 1, n3_go_prim, endpoint = False)

Y1_prim = np.real(chi0rr_prim[R1D_prim[0], R1D_prim[1], R1D_prim[2], :, R1D_prim[1], R1D_prim[2]])
Y2_prim = np.real(chi0rr_prim[R1D_prim[0], R1D_prim[1], R1D_prim[2], R1D_prim[0], :, R1D_prim[2]])
Y3_prim = np.real(chi0rr_prim[R1D_prim[0], R1D_prim[1], R1D_prim[2], R1D_prim[0], R1D_prim[1], :])

Y1R_prim = np.real(chi0rr_prim[:, R1D_prim[1], R1D_prim[2], R1D_prim[0], R1D_prim[1], R1D_prim[2]])
Y2R_prim = np.real(chi0rr_prim[R1D_prim[0], :, R1D_prim[2], R1D_prim[0], R1D_prim[1], R1D_prim[2]])
Y3R_prim = np.real(chi0rr_prim[R1D_prim[0], R1D_prim[1], :, R1D_prim[0], R1D_prim[1], R1D_prim[2]])

""" Y1_prim = np.real(chi0rr_prim[0, 0, 0, :, 0, 0])
Y2_prim = np.real(chi0rr_prim[0, 0, 0, 0, :, 0])
Y3_prim = np.real(chi0rr_prim[0, 0, 0, 0, 0, :])

Y1R_prim = np.real(chi0rr_prim[:, 0, 0, 0, 0, 0])
Y2R_prim = np.real(chi0rr_prim[0, :, 0, 0, 0, 0])
Y3R_prim = np.real(chi0rr_prim[0, 0, :, 0, 0, 0]) """

' Y1_prim = np.real(chi0rr_prim[0, 0, 0, :, 0, 0])\nY2_prim = np.real(chi0rr_prim[0, 0, 0, 0, :, 0])\nY3_prim = np.real(chi0rr_prim[0, 0, 0, 0, 0, :])\n\nY1R_prim = np.real(chi0rr_prim[:, 0, 0, 0, 0, 0])\nY2R_prim = np.real(chi0rr_prim[0, :, 0, 0, 0, 0])\nY3R_prim = np.real(chi0rr_prim[0, 0, :, 0, 0, 0]) '

In [43]:
trace1 = go.Scatter(x = X1_prim, y = Y1_prim, mode = "lines+markers", name = "Axe0")

trace2 = go.Scatter(x = X2_prim, y = Y2_prim, mode = "lines+markers", name = "Axe1" )

trace3 = go.Scatter(x = X3_prim, y = Y3_prim, mode = "lines+markers", name = "Axe2" )

trace4 = go.Scatter(x = X1R_prim, y = Y1R_prim, mode = "lines+markers", name = "Axe0K" )

trace5 = go.Scatter(x = X2R_prim, y = Y2R_prim, mode = "lines+markers", name = "Axe1K" )

trace6 = go.Scatter(x = X3R_prim, y = Y3R_prim, mode = "lines+markers", name = "Axe2K" )

data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = dict(title = '\chi^0(r, r\') with r_red ='+str([0, 0, 0])+' for different directions',
              xaxis= dict(title= 'Number of unit cells',ticklen= 5,zeroline= False),
             )
fig = go.Figure(dict(data = data, layout = layout))
fig.show()

In [45]:
R1D_slab = [0, 0, 0]

In [46]:
X1_slab = np.linspace(0, ngkpt_slab[0], n4_go_slab,endpoint = False)
X2_slab = np.linspace(0, ngkpt_slab[1], n5_go_slab,endpoint = False)
X3_slab = np.linspace(0, ngkpt_slab[2], n6_go_slab,endpoint = False)

X1R_slab = np.linspace(0, 1, n1_go_slab, endpoint = False)
X2R_slab = np.linspace(0, 1, n2_go_slab, endpoint = False)
X3R_slab = np.linspace(0, 1, n3_go_slab, endpoint = False)


Y1_slab = np.real(chi0rr_slab[R1D_slab[0], R1D_slab[1], R1D_slab[2], :, R1D_slab[1], R1D_slab[2]])
Y2_slab = np.real(chi0rr_slab[R1D_slab[0], R1D_slab[1], R1D_slab[2], R1D_slab[0], :, R1D_slab[2]])
Y3_slab = np.real(chi0rr_slab[R1D_slab[0], R1D_slab[1], R1D_slab[2], R1D_slab[0], R1D_slab[1], :])

Y1R_slab = np.real(chi0rr_slab[:, R1D_slab[1], R1D_slab[2], R1D_slab[0], R1D_slab[1], R1D_slab[2]])
Y2R_slab = np.real(chi0rr_slab[R1D_slab[0], :, R1D_slab[2], R1D_slab[0], R1D_slab[1], R1D_slab[2]])
Y3R_slab = np.real(chi0rr_slab[R1D_slab[0], R1D_slab[1], :, R1D_slab[0], R1D_slab[1], R1D_slab[2]])

""" Y1_slab = np.real(chi0rr_slab[0,0,0,:,0,0])
Y2_slab = np.real(chi0rr_slab[0,0,0,0,:,0])
Y3_slab = np.real(chi0rr_slab[0,0,0,0,0,:])

Y1R_slab = np.real(chi0rr_slab[:,0,0,0,0,0])
Y2R_slab = np.real(chi0rr_slab[0,:,0,0,0,0])
Y3R_slab = np.real(chi0rr_slab[0,0,:,0,0,0]) """

' Y1_slab = np.real(chi0rr_slab[0,0,0,:,0,0])\nY2_slab = np.real(chi0rr_slab[0,0,0,0,:,0])\nY3_slab = np.real(chi0rr_slab[0,0,0,0,0,:])\n\nY1R_slab = np.real(chi0rr_slab[:,0,0,0,0,0])\nY2R_slab = np.real(chi0rr_slab[0,:,0,0,0,0])\nY3R_slab = np.real(chi0rr_slab[0,0,:,0,0,0]) '

In [47]:
trace1 = go.Scatter(x = X1_slab, y = Y1_slab, mode = "lines+markers", name = "Axe0")

trace2 = go.Scatter(x = X2_slab, y = Y2_slab, mode = "lines+markers", name = "Axe1" )

trace3 = go.Scatter(x = X3_slab, y = Y3_slab, mode = "lines+markers", name = "Axe2" )

trace4 = go.Scatter(x = X1R_slab, y = Y1R_slab, mode = "lines+markers", name = "Axe0K" )

trace5 = go.Scatter(x = X2R_slab, y = Y2R_slab, mode = "lines+markers", name = "Axe1K" )

trace6 = go.Scatter(x = X3R_slab, y = Y3R_slab, mode = "lines+markers", name = "Axe2K" )

data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = dict(title = '\chi^0(r, r\') with r_red ='+str([0, 0, 0])+' for different directions',
              xaxis= dict(title= 'Number of unit cells',ticklen= 5,zeroline= False),
             )
fig = go.Figure(dict(data = data, layout = layout))
fig.show()

# Visualisation tool 3D

In [48]:
R3D_prim = [0, 0, 0]

In [50]:
XGChi0.Vis_tool(chi0rr_prim, R3D_prim, A_prim, B_prim, C_prim, ngkpt_prim, natom_prim, xred_prim, N=10000, isomin=1)

In [51]:
R3D_slab = [0, 0, 0]

In [52]:
XGChi0.Vis_tool(chi0rr_slab, R3D_slab, A_slab, B_slab, C_slab, ngkpt_slab, natom_slab, xred_slab, N=10000, isomin=1)

In [ ]:
# Visualisation tool 2D

In [53]:
radius = 1
charge_evol_prim = XGChi0.charge_evol(chi0rr_prim, radius)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [60]:
layout_prim = dict(title = 'Charge accumulation in a box of size '+str(radius)+' in the plane yz',
              xaxis= dict(title= 'z axis',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'y axis',ticklen= 5,zeroline= False),
             )
fig = go.Figure(data = go.Contour(z = np.real(charge_evol_prim[0, :, :])), layout = layout_prim)
fig.show()